In [1]:
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
import os
from PIL import Image
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import matplotlib.pyplot as plt
import math
import time
import random

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
TRANSFORM = transforms.Compose([transforms.Resize((84,84)),
                                transforms.ToTensor()
                               # transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
                               ])

In [4]:
class ImageDataset(data.Dataset):
    def __init__(self, image_list, label_list):
        self.data = image_list
        self.label = label_list
 
    def __getitem__(self, index):
        global TRANSFORM
        img = Image.open(self.data[index])#.conver('RGB')
        data = TRANSFORM(img.convert("RGB"))
        img.close()
        return data,torch.FloatTensor([self.label[index]])
 
    def __len__(self):
        return len(self.data)

In [5]:
root_dir=os.getcwd()
path_lt=os.path.join(root_dir,'dataset/lt/')
path_ult=os.path.join(root_dir,'dataset/ult/')
path_whole=os.path.join(root_dir,'dataset/whole/')
OUTPUT_PATH=os.path.join(root_dir,'output/')

In [6]:
dir_whole=[i for i in os.walk(path_whole)][0]
list_dir=list()
list_tag=list()
tem_index=dir_whole[2][:]
random.shuffle(tem_index)
#print(tem_index)
for i in tem_index:
    list_dir.append(os.path.join(dir_whole[0],i))
    list_tag.append(int(i[0]))
n=int(len(list_dir)*0.8)
train_data = ImageDataset(list_dir[:n],list_tag[:n])
validate_data = ImageDataset(list_dir[n:],list_tag[n:])


In [7]:
class VGG(nn.Module):#11
    def __init__(self):
        super(VGG, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, 5), 
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*18*18, 1024),
            nn.ReLU(),
            #nn.Linear(1024,512),
           # nn.ReLU(),
            nn.Linear(1024, 1),
            #nn.ReLU()
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [8]:
#cirterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [11]:
net=VGG()
optimizer = optim.SGD(net.parameters(), lr=0.001)
LOSS_FUNC = nn.BCELoss()

train_loader = data.DataLoader(train_data, 
                               batch_size=1, 
                               shuffle=True, 
                               num_workers=0,
                               drop_last=True,)
validate_loader = data.DataLoader(validate_data, batch_size=1, shuffle=False, num_workers=0,drop_last=True,)

In [13]:
time_start=time.time()
PROGRAM_START = time.time()
loss_list=list()
EPOCH=30
BATCH = len(train_loader)
m = len(validate_loader)
UPDATE = 5
for epoch in range(EPOCH):
    EPOCH_START = time.time()
    print("轮次 #{0}\\{1}:".format(epoch+1,EPOCH))
    
    for batch,(x,y) in enumerate(train_loader):
        optimizer.zero_grad()
       # print(x.shape,y)
        t = net(x)
      #  print(t)
        loss = LOSS_FUNC(t,y)
        loss_list.append(loss)
        if batch%10==0:
            print("batch {0}\\{1}: ".format(batch+1,BATCH) + "Loss = %.6f"%float(loss))
        loss.backward()
        optimizer.step()
    #保存每一epoch的模型
    with torch.no_grad():
        L = 0.
        E = 0.
        for batch,(x_,y_) in enumerate(validate_loader):
          #  print(y_)
           # print(x_)
            t = net(x_)
           # print(t)
            L += float(LOSS_FUNC(t,y_))
            #print(t)
            E += float((float(t[0][0])>0.5)!=y_)
        print("\t  Validation Loss = %.6f. Error Rate = %.3f%%"%(L/m,E*100/m))
        if((epoch+1)%UPDATE==0):
            torch.save(net.state_dict(),OUTPUT_PATH+"\\[{}]".format(epoch+1)+"-L(%.6f)E(%.3f).pt"%(L/m,E*100/m))
            torch.save(optimizer.state_dict(),OUTPUT_PATH+"\\[{}]".format(epoch+1)+"-L(%.6f)E(%.3f)-optimizer.pt"%(L/m,E*100/m))
    print("\t 完成第{0}轮".format(epoch+1)+" 耗时 %.4f s."%(time.time()-EPOCH_START)+
          " 总耗时 = %.4f s."%(time.time()-PROGRAM_START))


轮次 #1\30:
batch 1\480: Loss = 0.591116
batch 11\480: Loss = 0.968059
batch 21\480: Loss = 0.800377
batch 31\480: Loss = 0.610558
batch 41\480: Loss = 0.694613
batch 51\480: Loss = 0.593039
batch 61\480: Loss = 0.882689
batch 71\480: Loss = 0.699051
batch 81\480: Loss = 0.601024
batch 91\480: Loss = 0.729678
batch 101\480: Loss = 0.740710
batch 111\480: Loss = 0.668870
batch 121\480: Loss = 0.776083
batch 131\480: Loss = 0.607832
batch 141\480: Loss = 0.802152
batch 151\480: Loss = 0.619446
batch 161\480: Loss = 0.648362
batch 171\480: Loss = 0.492503
batch 181\480: Loss = 0.520644
batch 191\480: Loss = 0.408271
batch 201\480: Loss = 0.968329
batch 211\480: Loss = 0.433390
batch 221\480: Loss = 0.824662
batch 231\480: Loss = 0.735885
batch 241\480: Loss = 0.456022
batch 251\480: Loss = 0.436459
batch 261\480: Loss = 0.620071
batch 271\480: Loss = 0.702103
batch 281\480: Loss = 0.719851
batch 291\480: Loss = 0.604067
batch 301\480: Loss = 0.745959
batch 311\480: Loss = 0.871596
batch 321

batch 111\480: Loss = 0.929227
batch 121\480: Loss = 0.258717
batch 131\480: Loss = 0.551956
batch 141\480: Loss = 0.775970
batch 151\480: Loss = 0.251697
batch 161\480: Loss = 0.342020
batch 171\480: Loss = 0.325584
batch 181\480: Loss = 1.100846
batch 191\480: Loss = 0.833127
batch 201\480: Loss = 0.167762
batch 211\480: Loss = 1.745710
batch 221\480: Loss = 0.886819
batch 231\480: Loss = 0.670236
batch 241\480: Loss = 1.354788
batch 251\480: Loss = 0.108064
batch 261\480: Loss = 0.266226
batch 271\480: Loss = 0.087603
batch 281\480: Loss = 0.680312
batch 291\480: Loss = 0.725403
batch 301\480: Loss = 0.181748
batch 311\480: Loss = 0.475023
batch 321\480: Loss = 0.228175
batch 331\480: Loss = 0.543547
batch 341\480: Loss = 1.218340
batch 351\480: Loss = 0.355543
batch 361\480: Loss = 0.420103
batch 371\480: Loss = 0.518843
batch 381\480: Loss = 0.589886
batch 391\480: Loss = 0.591735
batch 401\480: Loss = 0.226885
batch 411\480: Loss = 0.828584
batch 421\480: Loss = 1.000409
batch 43

batch 211\480: Loss = 0.070128
batch 221\480: Loss = 1.179241
batch 231\480: Loss = 0.738501
batch 241\480: Loss = 0.741872
batch 251\480: Loss = 0.242438
batch 261\480: Loss = 1.287374
batch 271\480: Loss = 0.377080
batch 281\480: Loss = 0.056418
batch 291\480: Loss = 0.214006
batch 301\480: Loss = 0.056761
batch 311\480: Loss = 0.586166
batch 321\480: Loss = 0.251042
batch 331\480: Loss = 0.665495
batch 341\480: Loss = 0.791126
batch 351\480: Loss = 0.396657
batch 361\480: Loss = 0.634935
batch 371\480: Loss = 0.389928
batch 381\480: Loss = 0.582390
batch 391\480: Loss = 0.721924
batch 401\480: Loss = 0.209470
batch 411\480: Loss = 0.545941
batch 421\480: Loss = 0.922162
batch 431\480: Loss = 1.596514
batch 441\480: Loss = 0.243743
batch 451\480: Loss = 0.442667
batch 461\480: Loss = 0.202479
batch 471\480: Loss = 0.382884
	  Validation Loss = 0.503233. Error Rate = 24.167%
	 完成第11轮 耗时 13.2402 s. 总耗时 = 147.8283 s.
轮次 #12\30:
batch 1\480: Loss = 0.591545
batch 11\480: Loss = 0.572961


batch 311\480: Loss = 0.430983
batch 321\480: Loss = 0.222328
batch 331\480: Loss = 2.158133
batch 341\480: Loss = 0.276013
batch 351\480: Loss = 0.257292
batch 361\480: Loss = 0.090207
batch 371\480: Loss = 0.208626
batch 381\480: Loss = 0.007811
batch 391\480: Loss = 0.927061
batch 401\480: Loss = 0.806094
batch 411\480: Loss = 1.522825
batch 421\480: Loss = 0.381664
batch 431\480: Loss = 1.097764
batch 441\480: Loss = 0.108426
batch 451\480: Loss = 0.019335
batch 461\480: Loss = 0.699047
batch 471\480: Loss = 0.201686
	  Validation Loss = 0.504025. Error Rate = 20.833%
	 完成第16轮 耗时 13.3454 s. 总耗时 = 214.5572 s.
轮次 #17\30:
batch 1\480: Loss = 0.350901
batch 11\480: Loss = 0.372763
batch 21\480: Loss = 0.343875
batch 31\480: Loss = 0.925409
batch 41\480: Loss = 0.025632
batch 51\480: Loss = 0.428149
batch 61\480: Loss = 0.054652
batch 71\480: Loss = 0.400556
batch 81\480: Loss = 0.414658
batch 91\480: Loss = 0.576535
batch 101\480: Loss = 0.158285
batch 111\480: Loss = 0.882746
batch 12

batch 421\480: Loss = 0.402131
batch 431\480: Loss = 0.072561
batch 441\480: Loss = 0.254989
batch 451\480: Loss = 0.014962
batch 461\480: Loss = 0.811458
batch 471\480: Loss = 0.249734
	  Validation Loss = 0.511975. Error Rate = 21.667%
	 完成第21轮 耗时 13.4481 s. 总耗时 = 281.3608 s.
轮次 #22\30:
batch 1\480: Loss = 2.131779
batch 11\480: Loss = 0.000942
batch 21\480: Loss = 0.070174
batch 31\480: Loss = 0.007043
batch 41\480: Loss = 0.266884
batch 51\480: Loss = 0.481896
batch 61\480: Loss = 0.210215
batch 71\480: Loss = 0.237270
batch 81\480: Loss = 0.520523
batch 91\480: Loss = 0.303419
batch 101\480: Loss = 0.338547
batch 111\480: Loss = 0.092405
batch 121\480: Loss = 0.271837
batch 131\480: Loss = 0.318406
batch 141\480: Loss = 0.031298
batch 151\480: Loss = 0.082894
batch 161\480: Loss = 0.063745
batch 171\480: Loss = 0.080804
batch 181\480: Loss = 0.008366
batch 191\480: Loss = 0.273373
batch 201\480: Loss = 0.114573
batch 211\480: Loss = 0.045566
batch 221\480: Loss = 0.095116
batch 23

batch 11\480: Loss = 0.026288
batch 21\480: Loss = 0.341445
batch 31\480: Loss = 0.035951
batch 41\480: Loss = 0.329229
batch 51\480: Loss = 0.715056
batch 61\480: Loss = 0.001738
batch 71\480: Loss = 0.533818
batch 81\480: Loss = 0.006983
batch 91\480: Loss = 0.054333
batch 101\480: Loss = 0.041737
batch 111\480: Loss = 0.023655
batch 121\480: Loss = 0.144483
batch 131\480: Loss = 0.015285
batch 141\480: Loss = 0.001546
batch 151\480: Loss = 0.303101
batch 161\480: Loss = 0.510980
batch 171\480: Loss = 0.000318
batch 181\480: Loss = 0.037175
batch 191\480: Loss = 0.036787
batch 201\480: Loss = 0.029406
batch 211\480: Loss = 0.000806
batch 221\480: Loss = 0.017245
batch 231\480: Loss = 1.602858
batch 241\480: Loss = 0.369046
batch 251\480: Loss = 0.054220
batch 261\480: Loss = 0.294970
batch 271\480: Loss = 0.180460
batch 281\480: Loss = 0.001892
batch 291\480: Loss = 0.512881
batch 301\480: Loss = 0.806588
batch 311\480: Loss = 0.003288
batch 321\480: Loss = 0.129657
batch 331\480: Lo

In [16]:

dir_lt=[i for i in os.walk(path_lt)][0]
dir_ult=[i for i in os.walk(path_ult)][0]
#正样例
for i in range(len(dir_lt[2])):
    path_single_pic=os.path.join(path_lt,dir_lt[0]+dir_lt[2][i])
    #print(path_single_pic)
    img_sigle_pic=Image.open(path_single_pic)
    img_sigle_pic.save(os.path.join(path_whole,'1_'+str(i)+'.png'))

for i in range(len(dir_ult[2])):
    path_single_pic=os.path.join(path_ult,dir_ult[0]+dir_ult[2][i])
    #print(path_single_pic)
    img_sigle_pic=Image.open(path_single_pic)
    img_sigle_pic.save(os.path.join(path_whole,'0_'+str(i)+'.png'))
